# Import libraries

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tree Model").getOrCreate()

In [2]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.sql import functions as f 

# Load and verify data

In [3]:
data = spark.read.csv('College.csv',header = True, inferSchema = True)

In [4]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [5]:
data.head(3)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60),
 Row(School='Adelphi University', Private='Yes', Apps=2186, Accept=1924, Enroll=512, Top10perc=16, Top25perc=29, F_Undergrad=2683, P_Undergrad=1227, Outstate=12280, Room_Board=6450, Books=750, Personal=1500, PhD=29, Terminal=30, S_F_Ratio=12.2, perc_alumni=16, Expend=10527, Grad_Rate=56),
 Row(School='Adrian College', Private='Yes', Apps=1428, Accept=1097, Enroll=336, Top10perc=22, Top25perc=50, F_Undergrad=1036, P_Undergrad=99, Outstate=11250, Room_Board=3750, Books=400, Personal=1165, PhD=53, Terminal=66, S_F_Ratio=12.9, perc_alumni=30, Expend=8735, Grad_Rate=54)]

In [6]:
for item in data.head(1)[0]:
    print(item)

Abilene Christian University
Yes
1660
1232
721
23
52
2885
537
7440
3300
450
2200
70
78
18.1
12
7041
60


In [7]:
data.describe().show()

+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+------------------+
|summary|              School|Private|              Apps|            Accept|          Enroll|         Top10perc|         Top25perc|      F_Undergrad|      P_Undergrad|          Outstate|        Room_Board|             Books|          Personal|               PhD|          Terminal|         S_F_Ratio|       perc_alumni|          Expend|         Grad_Rate|
+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-------

In [8]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

# Data Preprocessing

In [9]:
assembler = VectorAssembler(inputCols =['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'],outputCol='features')
output = assembler.transform(data)

In [10]:
indexer = StringIndexer(inputCol='Private',outputCol='PrivateIndex')
indexed = indexer.fit(output).transform(output)


In [11]:
indexed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [12]:
final_data = indexed.select('features','PrivateIndex')

# Train Test split

In [13]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [14]:
train_data.show(2)

+--------------------+------------+
|            features|PrivateIndex|
+--------------------+------------+
|[81.0,72.0,51.0,3...|         0.0|
|[100.0,90.0,35.0,...|         0.0|
+--------------------+------------+
only showing top 2 rows



In [15]:
test_data.show(2)

+--------------------+------------+
|            features|PrivateIndex|
+--------------------+------------+
|[191.0,165.0,63.0...|         0.0|
|[193.0,146.0,55.0...|         0.0|
+--------------------+------------+
only showing top 2 rows



# Build and Evaluate Model 

In [16]:
binaryEvaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol = 'PrivateIndex')
multiclassEvaluator = MulticlassClassificationEvaluator(labelCol = 'PrivateIndex',metricName='accuracy')

## DecisionTreeClassifier

In [17]:
classifier = DecisionTreeClassifier(labelCol = 'PrivateIndex',featuresCol='features')
model = classifier.fit(train_data)
pred_data = model.transform(test_data)
pred_data.head(1)
print(binaryEvaluator.evaluate(pred_data))
print(multiclassEvaluator.evaluate(pred_data))

0.8965740388275599
0.92578125


## RandomForestClassifier

In [18]:
classifier = RandomForestClassifier(numTrees=150, labelCol = 'PrivateIndex',featuresCol='features')
model = classifier.fit(train_data)
pred_data = model.transform(test_data)
pred_data.head(1)
print(binaryEvaluator.evaluate(pred_data))
print(multiclassEvaluator.evaluate(pred_data))

0.9019794442329655
0.93359375


## GBTClassifier

In [19]:
classifier = GBTClassifier(labelCol = 'PrivateIndex',featuresCol='features')
model = classifier.fit(train_data)
pred_data = model.transform(test_data)
pred_data.head(1)
print(binaryEvaluator.evaluate(pred_data))
print(multiclassEvaluator.evaluate(pred_data))

0.8824895317853063
0.91796875
